# hvPlot.paths

```{eval-rst}
.. currentmodule:: hvplot

.. automethod:: hvPlot.paths
```

## Backend-specific styling options

```{eval-rst}
.. backend-styling-options:: paths
```

## Examples

### Basic paths plot from a DataFrame

A simple set of x/y coordinates is needed to generate a `paths` plot. While looking similarly to the output of a `line` plot, the object returned is a HoloViews [Path Element](https://holoviews.org/reference/elements/bokeh/Path.html) whose key dimensions are `'x'` and `'y'`, while for a `line` plot the returned object is a [Curve Element](https://holoviews.org/reference/elements/bokeh/Curve.html) with `'x'` as key dimension and `'y'` as value dimension. This models the independence between x and y, which is for example appropriate for representing a trajectory like a hurricane track.

In [ ]:
import hvplot.pandas  # noqa
import pandas as pd

df = pd.DataFrame({'x': [0, 1, 2, 3], 'y': [0, 0.75, 1.25, 1.5]})

df.hvplot.paths(x='x', y='y', aspect='equal')

### Vectorize `color` and `line_width`

This example sets [`color`](option-color) and `line_width`, both referencing the `'t'` dimension of this dataset: `color` with the `'t'` string, and `line_width` with a HoloViews [`dim`](https://holoviews.org/user_guide/Style_Mapping.html) object `hv.dim('t')**2` that squares each value of `'t'`. Each sub-line of the path has its color and width computed from these expressions.

In [ ]:
import holoviews as hv
import hvplot.pandas  # noqa
import pandas as pd

df = pd.DataFrame({'x': list(range(10)), 'y': list(range(10)), 't': list(range(1, 11))})

df.hvplot.paths(
    x='x', y='y', color='t', cmap='viridis',
    line_width=hv.dim('t')**2, aspect='equal', 
)

### From a GeoPandas dataset

The `paths` method can be used directly from [GeoPandas GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html) objects containing `LineString` or `MultiLineString` geometries.

In [ ]:
import hvplot.pandas  # noqa
import geopandas as gpd
from shapely.geometry import LineString

lines = [
    LineString([(-112.048, 33.451), (-112.03, 33.451)]),
    LineString([(-112.048, 33.459), (-112.03, 33.459)]),
    LineString([(-112.048, 33.466), (-112.03, 33.466)]),
    LineString([(-112.048, 33.44), (-112.048, 33.48)]),
    LineString([(-112.03, 33.44), (-112.03, 33.48)]),
]
categories = ['residential', 'residential', 'residential', 'arterial', 'arterial']
gdf = gpd.GeoDataFrame({'category': categories}, geometry=lines, crs='EPSG:4326')

gdf.hvplot.paths(
    color='category', cmap='category10',
    line_width=3, tiles=True, tiles_opts=dict(alpha=0.5),
)

In [ ]:
import hvplot.pandas  # noqa
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString

lines = [
    MultiLineString([
        LineString([(-112.048, 33.451), (-112.03, 33.451)]),
        LineString([(-112.048, 33.459), (-112.03, 33.459)]),
        LineString([(-112.048, 33.466), (-112.03, 33.466)]),
    ]),
    MultiLineString([
        LineString([(-112.048, 33.44), (-112.048, 33.48)]),
        LineString([(-112.03, 33.44), (-112.03, 33.48)]),
    ]),
]
categories = ['residential', 'arterial']
gdf = gpd.GeoDataFrame({'category': categories}, geometry=lines, crs='EPSG:4326')

gdf.hvplot.paths(
    color='category', cmap='category10',
    line_width=3, tiles=True, tiles_opts=dict(alpha=0.5),
)

### From a DataFrame containing geographic coordinates

*Example adapted from https://scitools.org.uk/cartopy/docs/latest/matplotlib/intro.html.*

Notice how the line in blue between New York and Delhi is not straight on a flat PlateCarree map, this is because the Geodetic coordinate system is a truly spherical coordinate system, where a line between two points is defined as the shortest path between those points on the globe rather than 2d Cartesian space.

In [ ]:
import cartopy.crs as ccrs
import hvplot.pandas  # noqa
import pandas as pd

df = pd.DataFrame({"city": ["NY", "Delhi"], "lon": [-75, 77.23], "lat": [43, 28.61]})

common_kwargs = dict(
    x="lon",
    y="lat",
    geo=True,
    project=True,
    projection=ccrs.GOOGLE_MERCATOR,
    global_extent=True
)
shortest_path = df.hvplot.paths(color="blue", crs=ccrs.Geodetic(), tiles=True, **common_kwargs)
straight_path = df.hvplot.paths(color="grey", line_dash="dashed", **common_kwargs)
shortest_path * straight_path

### From an Xarray Dataset

In [ ]:
import hvplot.xarray  # noqa
import numpy as np
import pandas as pd
import xarray as xr

time = pd.date_range("2025-09-01", periods=20, freq="6h")
lon = [-55.0]
lat = [16.0]
for _ in range(1, len(time)):
    lon.append(lon[-1] - np.random.uniform(0.3, 1.0))
    lat.append(lat[-1] + np.random.uniform(0.1, 0.5))
base_speed = np.linspace(40, 120, len(time))
noise = np.random.normal(0, 5, len(time))
wind_speed = base_speed + noise
ds = xr.Dataset(
    data_vars={"wind_speed": (["time"], wind_speed)},
    coords={
        "time": time,
        "lon": ("time", lon),
        "lat": ("time", lat)
    },
)

ds.hvplot.paths(
    x='lon', y='lat', color='wind_speed',
    hover_cols=['time'], line_width=5, tiles=True,
)